# 🍎 Health Calculator Agent Tutorial 🍏

Welcome to the **Health Calculator Agent** tutorial, where we'll showcase how to:
1. **Initialize** a project and use the Azure AI Foundry ecosystem
2. **Create an Agent** with **Code Interpreter** capabilities
3. **Perform BMI calculations** and **analyze nutritional data** with sample CSV files
4. **Generate** basic health insights and disclaimers

## Let's Dive In
We'll walk step-by-step, similar to our **Fun & Fit** sample, but with a focus on using **Code Interpreter** for numeric calculations and data analysis. Let's begin!


## 1. Initial Setup
We'll start by importing libraries, loading environment variables, and initializing an **AIProjectClient**. We'll also create a sample CSV for demonstration.


In [11]:
# Import required libraries
import os  # For environment variables and path operations
import time  # For adding delays if needed
from pathlib import Path  # For cross-platform path handling
import json
import pandas as pd

# Import Azure and utility libraries
from azure.identity import DefaultAzureCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # Main client for AI Projects
from azure.ai.agents.models import CodeInterpreterTool


def find_cred_json(start_path):
    current = Path(start_path)
    while current != current.parent:
        cred_file = current / 'cred.json'
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

start_dir = os.getcwd()  # Use current directory
file_path = find_cred_json(start_dir)

print(f"Found cred.json at: {file_path}")

try:
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)

    print("Project Connection String:", loaded_config['PROJECT_CONNECTION_STRING'])
    print("Tenant ID:", loaded_config['TENANT_ID'])
    print("Model Deployment ID:", loaded_config['MODEL_DEPLOYMENT_NAME'])

    project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=loaded_config["PROJECT_ENDPOINT"],
    )

    print("✅ Successfully initialized AIProjectClient")

except FileNotFoundError:
    print(f"❌ Could not find file at: {file_path}")
except json.JSONDecodeError:
    print("❌ File exists but contains invalid JSON")
except TypeError:
    print("❌ File path was None — cred.json not found in any parent directories.")


# Create sample CSV data for demonstration
def create_sample_data():
    try:
        data = {
            'Date': pd.date_range(start='2024-01-01', periods=7),
            'Calories': [2100, 1950, 2300, 2050, 1900, 2200, 2150],
            'Protein_g': [80, 75, 85, 78, 72, 82, 79],
            'Carbs_g': [250, 230, 270, 245, 225, 260, 255],
            'Fat_g': [70, 65, 75, 68, 63, 73, 71],
            'Fiber_g': [25, 22, 28, 24, 21, 26, 23]
        }
        df = pd.DataFrame(data)
        filename = "nutrition_data.csv"
        df.to_csv(filename, index=False)
        print(f"📄 Created sample data file: {filename}")
        return filename
    except Exception as e:
        print(f"❌ Error creating sample data: {e}")
        return None

sample_file = create_sample_data()

Found cred.json at: /workspaces/Azure-ai-foundry01/cred.json
Project Connection String: 
Tenant ID: 
Model Deployment ID: gpt-4o
✅ Successfully initialized AIProjectClient
📄 Created sample data file: nutrition_data.csv


## 2. Create Health Calculator Agent 👩‍💻
We'll upload our sample CSV and then create an agent with **Code Interpreter** enabled. This agent can read the file, run Python code, and return results and visualizations.


In [12]:
def create_health_calculator(file_path: str):
    """Create an agent with Code Interpreter for health/nutrition calculations."""
    try:
        # Upload the file – use "assistants" purpose (valid option)
        uploaded_file = project_client.agents.files.upload_and_poll(
            file_path=file_path,
            purpose="assistants"
        )
        print(f"✅ Uploaded CSV file, ID: {uploaded_file.id}")

        # Bind it with Code Interpreter
        code_tool = CodeInterpreterTool(file_ids=[uploaded_file.id])

        # Create the agent with tool enabled
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),
            name="health-calculator-agent",
            instructions=(
                "You are a health calculator agent that can:\n"
                "1. Calculate and interpret BMI\n"
                "2. Analyze nutrition data\n"
                "3. Generate charts/plots\n"
                "4. Always include medical disclaimers"
            ),
            tools=code_tool.definitions,
            tool_resources=code_tool.resources
        )
        print(f"🎉 Agent created, ID: {agent.id}")
        return agent, uploaded_file
    except Exception as e:
        print(f"❌ Error creating health calculator agent: {e}")
        return None, None

# Example setup
health_agent, uploaded_file = None, None
if 'sample_file' in globals() and sample_file:
    health_agent, uploaded_file = create_health_calculator(sample_file)

✅ Uploaded CSV file, ID: assistant-J3xpGNMeiWXmV8dcWAg5bd
🎉 Agent created, ID: asst_yvh1BeXVFLnmkZhKRZ2QFcrk


## 3. BMI Calculation with Code Interpreter
We'll create a thread for BMI calculations. We'll feed in the user's height/weight, and ask the agent to show how it calculates BMI, interpret the result, and always disclaim professional advice.


In [22]:
import time

def calculate_bmi_with_agent(agent, height_inches, weight_pounds):
    """Calculate BMI using the code interpreter agent."""
    try:
        # Create a new conversation thread
        thread = project_client.agents.threads.create()
        print(f"📝 Created thread for BMI calculation, ID: {thread.id}")

        # Construct user message
        user_text = (
            f"Calculate BMI for \n"
            f"Height: {height_inches} inches\n"
            f"Weight: {weight_pounds} pounds\n"
            "Please: \n"
            "1. Show calculation \n"
            "2. Interpret the result \n"
            "3. Include disclaimers \n"
        )

        msg = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_text
        )
        print(f"➕ Created BMI request message, ID: {msg.id}")

        # Create and run the agent
        run = project_client.agents.runs.create(
            thread_id=thread.id,
            agent_id=agent.id
        )
        print(f"▶️ Started run, ID: {run.id}, initial status: {run.status}")

        # Poll until completion
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.runs.get(
                thread_id=thread.id,
                run_id=run.id
            )

        print(f"🤖 BMI run finished with status: {run.status}")
        
        # Display results if successful
        if run.status == "completed":
            messages = project_client.agents.messages.list(thread_id=thread.id)
            message_list = list(messages)
            
            for message in message_list:
                if message.role == "assistant":
                    for content in message.content:
                        if hasattr(content, 'text') and content.text:
                            print(f"\n🤖 BMI Analysis:\n{content.text.value}")
        
        return thread, run

    except Exception as e:
        print(f"❌ Error during BMI calculation: {e}")
        return None, None

# Example usage:
if health_agent:
    bmi_thread, bmi_run = calculate_bmi_with_agent(health_agent, 70, 180)

📝 Created thread for BMI calculation, ID: thread_Wpdyy8FeU8G7iqzldrItzoCt
➕ Created BMI request message, ID: msg_tRlOQVK8pAWDJSSYEuvZTJ7s
▶️ Started run, ID: run_NIFdopGcKNugWQ4fV40SviLU, initial status: RunStatus.QUEUED
🤖 BMI run finished with status: RunStatus.COMPLETED

🤖 BMI Analysis:
### BMI Calculation Result:
\[
\text{BMI} = \frac{180 \times 703}{70^2} = 25.82
\]

### Interpretation of BMI:
According to the standard BMI categories:
- Underweight: BMI < 18.5
- Normal weight: BMI = 18.5–24.9
- Overweight: BMI = 25–29.9
- Obesity: BMI ≥ 30

Your BMI is **25.82**, which falls within the **Overweight** category.

---

### Important Medical Disclaimer:
- BMI is a general measure and does not account for muscle mass distribution, bone density, or individual variations.
- For a comprehensive health assessment, consult with a healthcare professional who may assess other metrics, including body fat percentage, dietary habits, and overall fitness levels.

🤖 BMI Analysis:
### BMI Calculatio

## 4. Nutrition Analysis
We'll create another thread where the user can ask the agent to analyze the **`nutrition_data.csv`** we uploaded. The agent can read the file, compute macros, produce charts, and disclaim that it's not offering personalized medical advice.


In [24]:
import time

def analyze_nutrition_data(agent):
    """Ask the agent to analyze the uploaded nutrition data."""
    try:
        thread = project_client.agents.threads.create()
        print(f"📝 Created thread for nutrition analysis, ID: {thread.id}")

        user_text = (
            "Analyze the CSV file with daily nutrition data.\n"
            "1. Compute average daily macros (calories, protein, carbs, fat, fiber).\n"
            "2. Create a chart to show trends.\n"
            "3. Discuss any insights or disclaimers.\n"
        )

        msg = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_text
        )
        print(f"➕ Created nutrition request message, ID: {msg.id}")

        # Create the run (not create_and_poll)
        run = project_client.agents.runs.create(
            thread_id=thread.id,
            agent_id=agent.id
        )
        print(f"▶️ Started run, ID: {run.id}, initial status: {run.status}")

        # Poll until completion
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.runs.get(
                thread_id=thread.id,
                run_id=run.id
            )

        print(f"🤖 Nutrition run finished with status: {run.status}")
        
        # Display results if successful
        if run.status == "completed":
            messages = project_client.agents.messages.list(thread_id=thread.id)
            message_list = list(messages)
            
            for message in message_list:
                if message.role == "assistant":
                    for content in message.content:
                        if hasattr(content, 'text') and content.text:
                            print(f"\n🤖 Nutrition Analysis:\n{content.text.value}")
                        elif hasattr(content, 'image_file'):
                            print(f"\n📊 Chart generated: {content.image_file.file_id}")
        elif run.status == "failed":
            print("❌ Run failed. Checking for error details...")
            if hasattr(run, 'last_error') and run.last_error:
                print(f"   Error Code: {run.last_error.code}")
                print(f"   Error Message: {run.last_error.message}")
        
        return thread, run

    except Exception as e:
        print(f"❌ Error analyzing nutrition data: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Example usage:
if health_agent:
    nutrition_thread, nutrition_run = analyze_nutrition_data(health_agent)

📝 Created thread for nutrition analysis, ID: thread_GsIcRoARTfJlIzKBSgHUfXwZ
➕ Created nutrition request message, ID: msg_aWabQzqfMUeKicrK3JIu199A
▶️ Started run, ID: run_bQeSp7GvPieVRcXFZ2xPh3JU, initial status: RunStatus.QUEUED
🤖 Nutrition run finished with status: RunStatus.COMPLETED

📊 Chart generated: assistant-QK97WgyE36oHWHX8wPsRg6

🤖 Nutrition Analysis:
### Insights:
The chart shows trends in daily macros over the 7-day period:
- **Calories:** There are moderate fluctuations, typically around the average of 2092.86 kcal. Spikes are seen on certain days, e.g., January 3 (2300 kcal).
- **Protein, Carbohydrates, and Fat:** These maintain relatively consistent levels throughout the week, with small variations.
- **Fiber:** There is slight variation but remains fairly steady across days, averaging 24.14 g.

### Discussion:
1. **Consistency:** Your nutritional intake appears balanced and does not exhibit extreme deviations, which supports stable dietary habits.
2. **Calories:** The v

## 5. Viewing Results & Visualizations 📊
The agent may produce text insights, disclaimers, and even images with charts. Let's fetch them from our threads!


In [25]:
def view_agent_responses(thread_id):
    try:
        messages = project_client.agents.messages.list(thread_id=thread_id)
        print("\n🔎 Agent Responses:")
        for msg in messages:
            if msg.role == "assistant" and msg.content:
                for c in msg.content:
                    if hasattr(c, "text"):
                        print("Response:", c.text.value, "\n")

        # Save image outputs if any
        for msg in messages:
            if msg.role == "assistant" and msg.content:
                for c in msg.content:
                    if hasattr(c, "image_file") and c.image_file:
                        file_id = c.image_file.file_id
                        outname = f"chart_{file_id}.png"
                        project_client.agents.files.download(file_id=file_id, output=outname)
                        print(f"🖼️ Saved image output: {outname}")

    except Exception as e:
        print(f"❌ Error viewing agent responses: {e}")

# Display BMI results
if 'bmi_thread' in globals() and bmi_thread and bmi_run:
    print("\n=== BMI Calculation Results ===")
    view_agent_responses(bmi_thread.id)

# Display nutrition analysis
if 'nutrition_thread' in globals() and nutrition_thread and nutrition_run:
    print("\n=== Nutrition Analysis Results ===")
    view_agent_responses(nutrition_thread.id)


=== BMI Calculation Results ===

🔎 Agent Responses:
Response: ### BMI Calculation Result:
\[
\text{BMI} = \frac{180 \times 703}{70^2} = 25.82
\]

### Interpretation of BMI:
According to the standard BMI categories:
- Underweight: BMI < 18.5
- Normal weight: BMI = 18.5–24.9
- Overweight: BMI = 25–29.9
- Obesity: BMI ≥ 30

Your BMI is **25.82**, which falls within the **Overweight** category.

---

### Important Medical Disclaimer:
- BMI is a general measure and does not account for muscle mass distribution, bone density, or individual variations.
- For a comprehensive health assessment, consult with a healthcare professional who may assess other metrics, including body fat percentage, dietary habits, and overall fitness levels. 

Response: ### BMI Calculation Process:

The formula to calculate BMI (Body Mass Index) is:

\[
\text{BMI} = \frac{\text{Weight (lbs)} \times 703}{\text{(Height (inches))}^2}
\]

Here:
- **Height** = 70 inches
- **Weight** = 180 pounds

Now let's calculate the 

## 6. Cleanup & Best Practices
We can remove our agent and sample data if desired. In production, you might keep them for repeated usage.

### Best Practices in a Nutshell
1. **Data Handling** – Validate input data, handle missing values, properly manage file attachments.
2. **Calculations** – Provide formula steps, disclaimers, limit scope to general wellness, remind user you're not a doctor.
3. **Visualizations** – Use clear labeling and disclaimers that charts are for educational demonstrations.
4. **Security** – Monitor usage, limit access to code interpreter if dealing with proprietary data.


In [31]:
import os

def cleanup_all_correct():
    """Cleanup function with correct method names."""
    
    # Delete uploaded file
    try:
        if 'uploaded_file' in globals() and uploaded_file:
            project_client.agents.files.delete(uploaded_file.id)
            print("🗑️ Deleted uploaded file from agent service.")
    except Exception as e:
        print(f"ℹ️ Uploaded file cleanup: {e}")
        # This is normal - file might already be deleted or auto-expired

    # Delete agent (correct method name: delete_agent)
    try:
        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health calculator agent.")
    except Exception as e:
        print(f"ℹ️ Agent cleanup: {e}")

    # Delete local CSV file
    try:
        if 'sample_file' in globals() and sample_file and os.path.exists(sample_file):
            os.remove(sample_file)
            print("🗑️ Deleted local sample CSV file.")
    except Exception as e:
        print(f"⚠️ Could not delete local file: {e}")

    print("✅ Cleanup completed!")

# Also create a function to list current resources
def list_current_resources():
    """List current agents and files."""
    try:
        print("📋 Current agents:")
        agents = project_client.agents.list_agents()
        agent_list = list(agents)
        if agent_list:
            for agent in agent_list:
                print(f"   - {agent.name} (ID: {agent.id})")
        else:
            print("   No agents found")
    except Exception as e:
        print(f"   Error listing agents: {e}")

    try:
        print("\n📋 Current files:")
        files = project_client.agents.files.list()
        file_list = list(files)
        if file_list:
            for file in file_list:
                print(f"   - {file.filename} (ID: {file.id})")
        else:
            print("   No files found")
    except Exception as e:
        print(f"   Error listing files: {e}")

# Run the corrected cleanup
print("=== Current Resources ===")
list_current_resources()

print("\n=== Running Cleanup ===")
cleanup_all_correct()

print("\n=== Resources After Cleanup ===")
list_current_resources()

=== Current Resources ===
📋 Current agents:
   - fun-fit-health-advisor (ID: asst_bmT90HZYgnM06MpwrEYtwK0a)
   - Agent904 (ID: asst_vHTAwHQrQ1f62EHQWrPVr2rJ)
   - fun-fit-health-advisor (ID: asst_fLv7hapk1BuVkjNNJsjyD6nm)

📋 Current files:
   Error listing files: 'str' object has no attribute 'filename'

=== Running Cleanup ===
ℹ️ Uploaded file cleanup: (None) No such File object: assistant-J3xpGNMeiWXmV8dcWAg5bd
Code: None
Message: No such File object: assistant-J3xpGNMeiWXmV8dcWAg5bd
ℹ️ Agent cleanup: (None) No assistant found with id 'asst_yvh1BeXVFLnmkZhKRZ2QFcrk'.
Code: None
Message: No assistant found with id 'asst_yvh1BeXVFLnmkZhKRZ2QFcrk'.
✅ Cleanup completed!

=== Resources After Cleanup ===
📋 Current agents:
   - fun-fit-health-advisor (ID: asst_bmT90HZYgnM06MpwrEYtwK0a)
   - Agent904 (ID: asst_vHTAwHQrQ1f62EHQWrPVr2rJ)
   - fun-fit-health-advisor (ID: asst_fLv7hapk1BuVkjNNJsjyD6nm)

📋 Current files:
   Error listing files: 'str' object has no attribute 'filename'


# Congratulations! 🎉
You now have a **Health Calculator Agent** with the **Code Interpreter** tool that can:
- Perform **BMI calculations** and disclaim that it's not a doctor.
- **Analyze** simple CSV-based nutrition data and produce insights + charts.
- Return images (charts) and text-based insights.

Happy (healthy) coding! 💪
